In [1]:
!pip install langchain langchain_community langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [21]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults # Internet access
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import BaseTool, StructuredTool, tool

GOOGLE_API_KEY = "YOUR_API_Key"
TAVILY_API_kEY = "YOUR_API_KEY"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_kEY

In [24]:
@tool
def list_files_in_folder(folder_path: str) -> str:
      """List available files in a folder."""
      try:
        files_and_dirs = os.listdir(folder_path)
        
        files = [f for f in files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

        if not files:
            return "No files found in the folder."

        return files
      except FileNotFoundError:
        return "The specified folder path does not exist."
      except Exception as e:
        return f"An error occurred: {e}"

In [34]:
#Tools for agent
tools = [list_files_in_folder, TavilySearchResults(max_results=1)]

In [35]:
# Using existing prompt for react agent
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [36]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [37]:
# buidling the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [38]:
# Create an pipeline with agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [39]:
agent_executor.invoke({"input": "what is autogen?"})

Action: tavily_search_results_json
Action Input: what is autogen[{'url': 'https://microsoft.github.io/autogen/docs/tutorial/introduction/', 'content': 'AutoGen is a framework that uses agents to enable complex workflows. Agents can be powered by models, code, human inputs or a combination of them. Learn how to install, create and use agents and conversations in AutoGen.'}]Action: list_files_in_folder
Action Input: /Users/michael/Desktop/autogenThe specified folder path does not exist.Final Answer: AutoGen is a framework that uses agents to enable complex workflows.

> Finished chain.


{'input': 'what is autogen?',
 'output': 'AutoGen is a framework that uses agents to enable complex workflows.'}

In [33]:
agent_executor.invoke({"input": "what are the files available on /content/sample_data folder"})

Action: list_files_in_folder
Action Input: /content/sample_data['anscombe.json', 'README.md', 'california_housing_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'mnist_test.csv']Final Answer: ['anscombe.json', 'README.md', 'california_housing_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'mnist_test.csv']

> Finished chain.


{'input': 'what are the files available on /content/sample_data folder',
 'output': "['anscombe.json', 'README.md', 'california_housing_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'mnist_test.csv']"}